In [300]:
import scrapy
import json
import requests
import requests_cache
import numpy as np
import pandas as pd
import seaborn as sns
import urlparse
import csv
import urllib
import os
import os.path
from bs4 import BeautifulSoup
import sys
from lxml import html
import lxml.html as lx

requests_cache.install_cache('demo_cache')

In [63]:
def Popular_Movies_search(year,pages,num):
    count = 0
    names = []
    
    for i in range(1,(pages+1)):
        url = "http://www.imdb.com/search/title?sort=moviemeter,asc&title_type=feature&year=" + str(year) + "," +str(year) + "&" + "page=" + str(i) + "&" + "ref_=adv_prv"        
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("p", class_="sort-num_votes-visible")
        vote = int(search_results[0].find_all('span')[1].text.replace(',',''))
        soup = BeautifulSoup(r.content, "html.parser")
        search_results = soup.find_all("div", class_="lister-item-content")
        

        for result in search_results:
            try:
                vote = int(result.find_all('p')[3].find_all('span')[1].text.replace(',',''))
                if vote < 10000:
                    continue
            except:
                continue
            header = result.find("h3", class_="lister-item-header")
            title = header.a
            b = title.get_text().encode('utf-8')
            names.append(b)
            count = count + 1
            if count == num:
                return names
            
    return names

In [321]:
def Movie_info_search(term):
    url = 'http://www.omdbapi.com/?t='+term.replace(' ','+')+'&plot=full&tomatoes=true'
    response = requests.get(url)
    try:
        js = response.json()
        map(js.pop, ['Plot','DVD','tomatoFresh','tomatoConsensus','tomatoMeter','tomatoRating','tomatoRotten','tomatoURL','tomatoUserMeter','tomatoUserRating','tomatoUserReviews','tomatoReviews'])
    except:
        return {}
    meta_url = 'http://www.metacritic.com/movie/'+term.replace(' ','-')
    return js

In [349]:
def Scoring_search(movie):
    imdb = Movie_info_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/' + str(imdb) + '/ratings?ref_=tt_ov_rt'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    res_2 = soup.find_all('td')
    
    rates = []
    try:
        for i in xrange(14):
            number = str(res_2[37 + 3*i].text.replace(u'\xa0',u''))
            rate = str(res_2[38 + 3*i].find_all('img')[0].text.replace(u'\xa0',u''))
            rates.append([number + ',' + rate])
    except:
        print 'sdfdsf'
    
    params = {'males':rates[0], 'females':rates[1],'<18':rates[2], 'males<18':rates[3],'females<18':rates[4], '18-29':rates[5],'males 18-29':rates[6], 'females 18-29':rates[7],'30-44':rates[8],'males 30-44':rates[9], 'females 30-44':rates[10], '45+':rates[11],'males 45+':rates[12], 'females 45+':rates[13]}
    return params

In [359]:
movie = 'la la land'
imdb = Movie_info_search(movie).values()[-1]
url = 'http://www.imdb.com/title/' + str(imdb) + '/ratings?ref_=tt_ov_rt'
r = requests.get(url)
Movie_info_search(movie).b

{u'Actors': u'Ryan Gosling, Emma Stone, Ami\xe9e Conn, Terry Walters',
 u'Awards': u'Nominated for 14 Oscars. Another 153 wins & 204 nominations.',
 u'BoxOffice': u'$118,306,924.00',
 u'Country': u'USA',
 u'Director': u'Damien Chazelle',
 u'Genre': u'Comedy, Drama, Musical',
 u'Language': u'English',
 u'Metascore': u'93',
 u'Poster': u'https://images-na.ssl-images-amazon.com/images/M/MV5BMzUzNDM2NzM2MV5BMl5BanBnXkFtZTgwNTM3NTg4OTE@._V1_SX300.jpg',
 u'Production': u'Liongate Films',
 u'Rated': u'PG-13',
 u'Released': u'25 Dec 2016',
 u'Response': u'True',
 u'Runtime': u'128 min',
 u'Title': u'La La Land',
 u'Type': u'movie',
 u'Website': u'http://www.lalaland.movie/',
 u'Writer': u'Damien Chazelle',
 u'Year': u'2016',
 u'imdbID': u'tt3783958',
 u'imdbRating': u'8.5',
 u'imdbVotes': u'165,257',
 u'tomatoImage': u'certified'}

In [132]:
def Comment_search(movie,gender):
    comments = []
    imdb = imdb = Movie_info_search(movie).values()[-1]
    url = 'http://www.imdb.com/title/'+str(imdb)+'/reviews?filter=custom&spoiler=&vote_min=&vote_max=&gender='+gender+'&age_min=&age_max=&country=&order=score&custom=1'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    div = soup.find('div', attrs = {'id' : 'tn15content'}).find_all('p')
    for contents in div:
        
        comments.append(contents.text.replace('\n',' '))
    
    return comments
    

In [341]:
films_name = []
films_data = pd.DataFrame()
for year in xrange(0,15):
    films_name.append(Popular_Movies_search(year+2002,10,100))
    films_data = pd.concat([films_data,pd.DataFrame.from_dict(Movie_info_search(i) for i in films_name[year])])
    


In [367]:
films_data.reset_index(drop = True)
films_data.to_csv('Films_data.csv',encoding = 'utf-8')

,Actors,Awards,BoxOffice,Country,Director,Genre,Language,Metascore,Poster,Production,...,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,tomatoImage,totalSeasons
0,"Naomi Watts, Martin Henderson, David Dorfman, ...",14 wins & 11 nominations.,"$128,579,698.00","USA, Japan",Gore Verbinski,"Horror, Mystery",English,57,https://images-na.ssl-images-amazon.com/images...,DreamWorks SKG,...,The Ring,movie,http://www.ring-themovie.com,"Ehren Kruger (screenplay), Kôji Suzuki (novel)",2002,tt0298130,7.1,"262,733",fresh,NaN
1,"Leonardo DiCaprio, Tom Hanks, Christopher Walk...",Nominated for 2 Oscars. Another 12 wins & 39 n...,"$164,400,000.00","USA, Canada",Steven Spielberg,"Biography, Crime, Drama","English, French",76,https://images-na.ssl-images-amazon.com/images...,DreamWorks SKG,...,Catch Me If You Can,movie,http://www.dreamworks.com/catchthem/jump2.html,"Jeff Nathanson (screenplay), Frank Abagnale Jr...",2002,tt0264464,8.0,"564,278",certified,NaN
2,"Leonardo DiCaprio, Daniel Day-Lewis, Cameron D...",Nominated for 10 Oscars. Another 49 wins & 118...,"$77,605,296.00","USA, Italy",Martin Scorsese,"Crime, Drama","English, Irish, Chinese, Latin",72,https://images-na.ssl-images-amazon.com/images...,Miramax Films,...,Gangs of New York,movie,http://www.gangsofnewyork.com/,"Jay Cocks (story), Jay Cocks (screenplay), Ste...",2002,tt0217505,7.5,"327,657",certified,NaN
3,"Tyler Hoechlin, Rob Maxey, Liam Aiken, Jennife...",Won 1 Oscar. Another 21 wins & 76 nominations.,"$104,054,514.00",USA,Sam Mendes,"Crime, Drama, Thriller",English,72,https://images-na.ssl-images-amazon.com/images...,DreamWorks SKG,...,Road to Perdition,movie,http://www.roadtoperdition.com/,"Max Allan Collins (graphic novel), Richard Pie...",2002,tt0257044,7.7,"206,372",certified,NaN
4,"Tobey Maguire, Willem Dafoe, Kirsten Dunst, Ja...",Nominated for 2 Oscars. Another 14 wins & 56 n...,"$403,706,375.00",USA,Sam Raimi,"Action, Adventure",English,73,https://images-na.ssl-images-amazon.com/images...,Columbia Pictures,...,Spider-Man,movie,http://spiderman.sonypictures.com/,"Stan Lee (Marvel comic book), Steve Ditko (Mar...",2002,tt0145487,7.3,"562,630",certified,NaN
5,"Alexandre Rodrigues, Leandro Firmino, Phellipe...",Nominated for 4 Oscars. Another 65 wins & 37 n...,N/A,"Brazil, France","Fernando Meirelles, Kátia Lund","Crime, Drama",Portuguese,79,https://images-na.ssl-images-amazon.com/images...,Miramax Films,...,City of God,movie,http://www.miramax.com/movie/city-of-god,"Paulo Lins (novel), Bráulio Mantovani (screenp...",2002,tt0317248,8.7,"557,588",certified,NaN
6,"Daniel Radcliffe, Rupert Grint, Emma Watson, R...",Nominated for 3 BAFTA Film Awards. Another 11 ...,"$261,835,892.00","UK, USA, Germany",Chris Columbus,"Adventure, Family, Fantasy",English,63,https://images-na.ssl-images-amazon.com/images...,Warner Bros. Pictures,...,Harry Potter and the Chamber of Secrets,movie,http://harrypotter.warnerbros.com/,"J.K. Rowling (novel), Steve Kloves (screenplay)",2002,tt0295297,7.4,"410,363",certified,NaN
7,"Bruce Allpress, Sean Astin, John Bach, Sala Baker",Won 2 Oscars. Another 114 wins & 127 nominations.,"$339,700,000.00","USA, New Zealand",Peter Jackson,"Adventure, Drama, Fantasy","English, Sindarin, Old English",88,https://images-na.ssl-images-amazon.com/images...,New Line Cinema,...,The Lord of the Rings: The Two Towers,movie,http://www.lordoftherings.net/,"J.R.R. Tolkien (novel), Fran Walsh (screenplay...",2002,tt0167261,8.7,"1,148,935",certified,NaN
8,"Ryan McCluskey, Oscar Pearce, Indra Ové, Anna ...",6 nominations.,"$39,532,308.00","UK, Germany, France, USA",Paul W.S. Anderson,"Action, Horror, Sci-Fi",English,33,https://images-na.ssl-images-amazon.com/images...,Columbia Tristar Pictures,...,Resident Evil,movie,http://www.spe.sony.com/movies/residentevil,Paul W.S. Anderson,2002,tt0120804,6.7,"205,614",rotten,NaN
9,"Taye Diggs, Cliff Saunders, Catherine Zeta-Jon...",Won 6 Oscars. Another 49 wins & 121 nominations.,N/A,"USA, Germany, Canada",Rob Marshall,"Comedy, 